In [1]:
import datetime as dt
import requests
import random
import time
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

import warnings

# Routine Update for METS Online

In [4]:
file_path = ".\export-data-malaysia.csv"
old_df = pd.read_csv(file_path)
old_df

,YEAR,FOOD,BEVERAGES AND TOBACCO,"CRUDE MATERIALS, INEDIBLE","MINERAL FUELS, LUBRICANTS, ETC.",ANIMAL AND VEGETABLE OILS AND FATS,CHEMICALS,MANUFACTURED GOODS,MACHINERY & TRANSPORT EQUIPMENT,MISCELLANEOUS MANUFACTURED ARTICLES,MISCELLANEOUS TRANSACTIONS AND COMMODITIES
0,Jan 2000,491864874,92332416,863994681,2843136683,1116946684,880219925,1927880545,14958386186,2026711654,229883181
1,Feb 2000,425641394,77214349,689603279,2881747034,1099756943,1072302224,1910675351,16986581083,1922449106,217203495
2,Mar 2000,553169207,100950924,901427068,3188319069,1254551970,1086130053,2133213423,20139482974,2368704586,262696514
3,Apr 2000,509671983,98515223,897853450,2455441551,1116557420,1087280719,2174508072,17989050119,2333513182,240322014
4,May 2000,553327793,111185482,909523666,2651869264,1008684467,1231347129,2291918346,18776956887,2480702004,253977343
...,...,...,...,...,...,...,...,...,...,...,...
277,Feb 2023,3539279631,222258336,2346324739,20323875331,5629973235,9036890095,10081396919,49850907614,10783053535,454585466
278,Mar 2023,4044164918,326561786,2715999243,21429738762,7647033259,9950967381,11464714044,57828441926,13401993529,858633228
279,Apr 2023,3251410911,229753325,1946644340,16984271124,5828048670,7583766017,10194652518,49009132271,9527059374,638171651
280,May 2023,3898340823,270752156,2454013437,21965803909,5767124163,9214928157,10424167008,52998110557,11859102111,657960595
